In [2]:
# ===== CRITICAL: 모든 캐시 초기화 =====
import sys
import importlib

print("🔄 Clearing module cache...")
# 현재 로드된 모든 모듈 제거
for module_name in list(sys.modules.keys()):
    if any(x in module_name for x in ['SEPN', 'DY_PS', 'MaskDataset', 'hybrid_encoder', 'Hungrian_match']):
        print(f"  Removing: {module_name}")
        del sys.modules[module_name]

print("✓ Cache cleared")

# ===== Core Libraries =====
import os
import random
import time
import math
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment

# 현재 디렉토리를 경로에 추가
sys.path.insert(0, '/data2/project/2025summer/yjh0913/JSFT_DETR')

# ===== PyTorch =====
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# ===== Torchvision =====
import torchvision.models as models
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.ops import generalized_box_iou, box_iou

# ===== External Modules Import (Fresh Load) =====
print("\n📦 Loading modules from fresh...")
from MaskDataset import MaskDataset
print("  ✓ MaskDataset loaded")

from SEPN import ResNet18Backbone, SEPN
print("  ✓ SEPN modules loaded")

from DY_PS import DySample, PSConv
print("  ✓ DY_PS modules loaded")

from hybrid_encoder import AIFI, CCFM, HybridEncoderBlock
print("  ✓ Hybrid Encoder modules loaded")

from Hungrian_match import (
    box_cxcywh_to_xyxy,
    HungarianMatcher,
    RTDETRDetection,
    SetCriterion
)
print("  ✓ Hungarian Matcher modules loaded")

# 기본 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n✓ Using device: {device}")
print("✓ All modules imported successfully\n")

# 테스트: ResNet18Backbone 생성 시도
print("🧪 Testing ResNet18Backbone creation...")
try:
    test_backbone = ResNet18Backbone()
    print("✓ ResNet18Backbone created successfully!")
except Exception as e:
    print(f"❌ Error creating ResNet18Backbone: {e}")

# 테스트: GAAM 생성 시도
print("🧪 Testing GAAM creation...")
try:
    from SEPN import GAAM
    test_gaam = GAAM(256, 128)
    print("✓ GAAM created successfully!")
except Exception as e:
    print(f"❌ Error creating GAAM: {e}")

# -----------------------------
# 1. collate_fn (기존 그대로)
# -----------------------------
def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch], dim=0)
    targets = [b[1] for b in batch]
    return imgs, targets


# -----------------------------
# 2. 이미지 경로 수집
# -----------------------------
def get_image_paths(img_dir, exts=(".jpg", ".png", ".jpeg", ".bmp")):
    return [
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(exts)
    ]


img_dir = "/data2/project/2025summer/yjh0913/DB-1/Images"      # 네 이미지 폴더
mask_dir = "/data2/project/2025summer/yjh0913/DB-1/Masks"      # 네 마스크 폴더

all_img_paths = get_image_paths(img_dir)
print(f"Total images: {len(all_img_paths)}")


# -----------------------------
# 3. 8:2 분할 (하지만 train에는 8만 사용)
# -----------------------------
random.seed(42)
random.shuffle(all_img_paths)

split_idx = int(len(all_img_paths) * 0.8)
train_img_paths = all_img_paths[:split_idx]
val_img_paths   = all_img_paths[split_idx:]   # ← 만들어두지만 train에는 안 씀

print(f"Train images (80%): {len(train_img_paths)}")
print(f"Val images (20%): {len(val_img_paths)}")


# -----------------------------
# 4. Train Dataset (8만 사용)
# -----------------------------
train_dataset = MaskDataset(
    img_paths=train_img_paths,
    mask_dir=mask_dir,
    img_size=640
)

train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

print("\n✅ Data loading complete!")


🔄 Clearing module cache...
✓ Cache cleared

📦 Loading modules from fresh...
  ✓ MaskDataset loaded
  ✓ SEPN modules loaded
  ✓ DY_PS modules loaded
  ✓ Hybrid Encoder modules loaded
  ✓ Hungarian Matcher modules loaded

✓ Using device: cuda
✓ All modules imported successfully

🧪 Testing ResNet18Backbone creation...
✓ ResNet18Backbone created successfully!
🧪 Testing GAAM creation...
✓ GAAM created successfully!
Total images: 1195
Train images (80%): 956
Val images (20%): 239

✅ Data loading complete!


In [3]:
class JFSTDETR(nn.Module):
    def __init__(
        self,
        num_classes=1,
        hidden_dim=256,
        num_queries=100
    ):
        super().__init__()

        # Backbone
        self.backbone = ResNet18Backbone()

        # SEPN
        self.sepn = SEPN()

        # Channel alignment
        self.proj3 = nn.Conv2d(128, hidden_dim, 1)
        self.proj4 = nn.Conv2d(256, hidden_dim, 1)
        self.proj5 = nn.Conv2d(512, hidden_dim, 1)

        # DySample (동적 샘플링)
        self.refine = DySample(hidden_dim)

        # Hybrid Encoder
        self.encoder = HybridEncoderBlock(hidden_dim)

        # Detection head
        self.detector = RTDETRDetection(
            hidden_dim=hidden_dim,
            num_classes=num_classes,
            num_queries=num_queries
        )

    def forward(self, x):
        # Backbone
        p2, p3, p4, p5 = self.backbone(x)

        # SEPN
        p3, p4, p5 = self.sepn(p2, p3, p4, p5)

        # Channel align
        p3 = self.proj3(p3)
        p4 = self.proj4(p4)
        p5 = self.proj5(p5)

        # DySample (각 scale별로 독립적으로 2배 업샘플링)
        p3 = self.refine(p3)
        p4 = self.refine(p4)
        p5 = self.refine(p5)

        # Hybrid Encoder
        p3, p4, p5 = self.encoder(p3, p4, p5)

        # Flatten (RT-DETR 방식)
        B, C, H3, W3 = p3.shape
        mem3 = p3.flatten(2).permute(0, 2, 1)
        mem4 = p4.flatten(2).permute(0, 2, 1)
        mem5 = p5.flatten(2).permute(0, 2, 1)
        memory = torch.cat([mem3, mem4, mem5], dim=1)

        # Detection
        return self.detector(memory)


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = JFSTDETR(
    num_classes=1,
    hidden_dim=256,
    num_queries=100
).to(device)

# Xavier uniform initialization (??? ??)
for m in model.modules():
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

matcher = HungarianMatcher(
    cost_class=1,
    cost_bbox=5,
    cost_giou=2
)

criterion = SetCriterion(
    num_classes=1,
    matcher=matcher,
    weight_dict={
        "loss_ce": 1,
        "loss_bbox": 5,
        "loss_giou": 2
    }
)

base_lr = 1e-4
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=base_lr,
    weight_decay=1e-4
)

warmup_epochs = 3
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-7)


In [ ]:
import os
import torch

# -------------------------
# ?? ??
# -------------------------
if 'model' not in globals():
    raise NameError("? Model not initialized")

if 'train_loader' not in globals():
    raise NameError("? train_loader not initialized")

if 'criterion' not in globals():
    raise NameError("? criterion not initialized")

if 'optimizer' not in globals():
    raise NameError("? optimizer not initialized")

print("? Model / DataLoader / Criterion / Optimizer confirmed")

device = next(model.parameters()).device

# -------------------------
# ?? ??
# -------------------------
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)

best_loss = float("inf")
num_epochs = 100

# =========================
# TRAIN LOOP
# =========================
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    sum_ce = 0.0
    sum_bbox = 0.0
    sum_giou = 0.0
    nan_count = 0
    empty_batches = 0
    valid_batches = 0

    if epoch < warmup_epochs:
        warmup_lr = base_lr * (epoch + 1) / warmup_epochs
        for group in optimizer.param_groups:
            group["lr"] = warmup_lr

    for batch_idx, (imgs, targets) in enumerate(train_loader):
        imgs = imgs.to(device)
        targets = [
            {
                "boxes": t["boxes"].to(device),
                "labels": t["labels"].to(device)
            }
            for t in targets
        ]

        if sum(t["boxes"].shape[0] for t in targets) == 0:
            empty_batches += 1

        # -----------------
        # forward
        # -----------------
        outputs = model(imgs)
        loss, loss_dict = criterion(outputs, targets)

        # NaN / Inf ??
        if not torch.isfinite(loss):
            nan_count += 1
            optimizer.zero_grad()
            continue

        # -----------------
        # backward
        # -----------------
        optimizer.zero_grad()
        loss.backward()

        # ?? Transformer ???
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item()
        sum_ce += loss_dict["loss_ce"].item()
        sum_bbox += loss_dict["loss_bbox"].item()
        sum_giou += loss_dict["loss_giou"].item()
        valid_batches += 1

    if epoch >= warmup_epochs:
        scheduler.step()

    # -----------------
    # epoch log
    # -----------------
    denom = max(valid_batches, 1)
    avg_loss = total_loss / denom
    avg_ce = sum_ce / denom
    avg_bbox = sum_bbox / denom
    avg_giou = sum_giou / denom

    print(
        f"[Epoch {epoch:03d}] "
        f"Loss: {avg_loss:.4f} | "
        f"CE: {avg_ce:.3f} | "
        f"BBox: {avg_bbox:.3f} | "
        f"GIoU: {avg_giou:.3f} | "
        f"Empty GT batches: {empty_batches} | "
        f"NaN batches: {nan_count}"
    )

    # -----------------
    # best model ??
    # -----------------
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": best_loss
            },
            os.path.join(save_dir, "jfst_detr_best.pth")
        )
        print(f"? Saved best model (loss={best_loss:.4f})")

print("
" + "=" * 50)
print("?? Training completed successfully!")
print("=" * 50)


✅ Model / DataLoader / Criterion / Optimizer confirmed
[Epoch 000] Loss: 0.0000 | CE: nan | BBox: nan | GIoU: nan | Skipped batches: 478 | NaN batches: 478
✅ Saved best model (loss=0.0000)


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torchvision.ops import box_iou

# ===============================
# box utils
# ===============================
def cxcywh_to_xyxy(boxes):
    cx, cy, w, h = boxes.unbind(-1)
    return torch.stack([
        cx - w / 2,
        cy - h / 2,
        cx + w / 2,
        cy + h / 2
    ], dim=-1)

# ===============================
# postprocess (DETR style)
# ===============================
def postprocess(pred_logits, pred_boxes, score_thresh=0.5):
    probs = F.softmax(pred_logits, dim=-1)
    scores, _ = probs[..., :-1].max(dim=-1)  # remove background
    keep = scores > score_thresh
    return pred_boxes[keep], scores[keep]

# ===============================
# TP / FP / FN
# ===============================
def calc_tp_fp_fn(pred_boxes, gt_boxes, iou_thresh=0.5):
    if len(pred_boxes) == 0:
        return 0, 0, len(gt_boxes)

    ious = box_iou(pred_boxes, gt_boxes)
    tp = 0
    matched = set()

    for i in range(len(pred_boxes)):
        max_iou, idx = ious[i].max(dim=0)
        if max_iou >= iou_thresh and idx.item() not in matched:
            tp += 1
            matched.add(idx.item())

    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp
    return tp, fp, fn

# ===============================
# AP (mAP@0.5)
# ===============================
def compute_ap(recalls, precisions):
    recalls = np.concatenate(([0.], recalls, [1.]))
    precisions = np.concatenate(([0.], precisions, [0.]))

    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = max(precisions[i - 1], precisions[i])

    idx = np.where(recalls[1:] != recalls[:-1])[0]
    return np.sum((recalls[idx + 1] - recalls[idx]) * precisions[idx + 1])

# ===============================
# evaluation
# ===============================
@torch.no_grad()
def evaluate(model, dataloader, device, score_thresh=0.5, iou_thresh=0.5):
    model.eval()
    total_tp = total_fp = total_fn = 0
    precisions, recalls = [], []

    for imgs, targets in dataloader:
        imgs = imgs.to(device)
        outputs = model(imgs)

        for i in range(len(imgs)):
            pb, _ = postprocess(
                outputs["pred_logits"][i],
                outputs["pred_boxes"][i],
                score_thresh
            )

            gt = targets[i]["boxes"]
            pb = cxcywh_to_xyxy(pb).cpu()
            gt = cxcywh_to_xyxy(gt).cpu()

            tp, fp, fn = calc_tp_fp_fn(pb, gt, iou_thresh)

            total_tp += tp
            total_fp += fp
            total_fn += fn

            p = tp / (tp + fp + 1e-6)
            r = tp / (tp + fn + 1e-6)
            precisions.append(p)
            recalls.append(r)

    precision = total_tp / (total_tp + total_fp + 1e-6)
    recall    = total_tp / (total_tp + total_fn + 1e-6)
    f1        = 2 * precision * recall / (precision + recall + 1e-6)
    map50     = compute_ap(np.array(recalls), np.array(precisions))

    return precision, recall, f1, map50

# ===============================
# RUN (학습된 모델로 평가)
# ===============================
print("\n📊 Evaluation on validation set...")

# val_img_paths에서 DataLoader 생성
val_dataset = MaskDataset(
    img_paths=val_img_paths,
    mask_dir=mask_dir,
    img_size=640
)
val_loader = DataLoader(
    val_dataset,
    batch_size=1,  # 메모리 효율을 위해 1로 설정
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

print(f"Validation dataset size: {len(val_dataset)}")

# 학습된 모델 사용 (새로 만들지 않음)
precision, recall, f1, map50 = evaluate(
    model,  # ← 이미 학습된 모델 재사용
    val_loader,
    device,
    score_thresh=0.5,
    iou_thresh=0.5
)

print("\n" + "="*50)
print("📈 EVALUATION RESULTS")
print("="*50)
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")
print(f"mAP@0.5   : {map50:.4f}")
print("="*50)
